In [3]:
import numpy as np
import pandas as pd
import pandapower as pp
import pandapower.networks as pn
import matplotlib.pyplot as plt
from pandapower.converter import to_ppc
from pandapower.pypower.makeYbus import makeYbus

pd.set_option('display.float_format', lambda x: '{:.4f}'.format(x))

# Crear la red a analizar
red2 = pp.create_empty_network()

# Creación de las barras del sistema
barra_1 = pp.create_bus(red2, vn_kv=110, name="Barra 1")
barra_2 = pp.create_bus(red2, vn_kv=220, name="Barra 2")
barra_1A = pp.create_bus(red2, vn_kv=220, name="Barra 1A")
barra_1B = pp.create_bus(red2, vn_kv=220, name="Barra 1B")
barra_2A = pp.create_bus(red2, vn_kv=220, name="Barra 2A")
barra_2B = pp.create_bus(red2, vn_kv=220, name="Barra 2B")
barra_3A = pp.create_bus(red2, vn_kv=220, name="Barra 3A")

# Añadir transformador entre barra 1 y 2
pp.create_transformer(red2, barra_2, barra_1, std_type="100 MVA 220/110 kV")

# Añadir el generador en la barra 1 y que dicha barra sea la referencia
pp.create_ext_grid(red2, barra_1, va_pu=1.0, name="Slack")

# Crear las líneas
pp.create_line(red2, barra_2, barra_1A, length_km=10.0, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", name="L2-1A")
pp.create_line(red2, barra_2, barra_1B, length_km=10.0, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", name="L2-1B")
pp.create_line(red2, barra_1A, barra_2A, length_km=15.0, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", name="L1A-2A")
pp.create_line(red2, barra_2A, barra_3A, length_km=20.0, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", name="L2A-3A")
pp.create_line(red2, barra_1B, barra_2B, length_km=30.0, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", name="L1B-2B")
pp.create_line(red2, barra_2B, barra_3A, length_km=15.0, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", name="L2B-3A")

# Crear las cargas
pp.create_load(red2, barra_1A, p_mw=30.0, q_mvar=20.0, name="Carga Barra 1A")
pp.create_load(red2, barra_2A, p_mw=52.5, q_mvar=35.0, name="Carga Barra 2A")
pp.create_load(red2, barra_3A, p_mw=22.5, q_mvar=15.0, name="Carga Barra 3A")
pp.create_load(red2, barra_1B, p_mw=15.0, q_mvar=10.0, name="Carga Barra 1B")
pp.create_load(red2, barra_2B, p_mw=90.0, q_mvar=60.0, name="Carga Barra 2B")

# Ejecutar el flujo de carga
pp.runpp(red2, algorithm='nr', numba=False, max_iteration=1000)

# Imprimir los resultados
red2.res_line.index = red2.line.index
red2.res_bus.index = red2.bus.index

print(red2.res_bus)
print(red2.res_line)

ppc = to_ppc(red2, init='flat')

# Obtener los datos de los buses y ramas
baseMVA = ppc["baseMVA"]
bus = ppc["bus"]
branch = ppc["branch"]

# Crear la matriz Y-bus
Ybus, Yf, Yt = makeYbus(baseMVA, bus, branch)

# Convertir la matriz Y-bus a formato denso
Ybus_dense = np.array(Ybus.todense())

# Convertir la matriz densa a una matriz de NumPy
Ybus_np = np.array(Ybus_dense)

# Aproximar los términos de la matriz a 3 decimales
Ybus_rounded = np.round(Ybus_np, decimals=3)

# Mostrar la matriz Y-bus densa en formato NumPy, aproximada a 3 decimales
print("Matriz Y-bus (NumPy) aproximada a 3 decimales:")
print(Ybus_rounded)

# Definir los parámetros y variables iniciales
V = np.ones(len(bus))  # Vector de tensiones iniciales (modulo)
theta = np.zeros(len(bus))  # Vector de ángulos de fase iniciales (en radianes)
max_iter = 10000  # Máximo número de iteraciones
tolerance = 1e-3  # Tolerancia para la convergencia

# Calcular P_spec y Q_spec
P_spec = np.zeros(len(bus))
Q_spec = np.zeros(len(bus))

for idx, bus_data in enumerate(bus):
    if bus_data[1] == 1:  # Si la barra es PV (Generador)
        P_spec[idx] = bus_data[2] / baseMVA
    elif bus_data[1] == 2:  # Si la barra es PQ (Carga)
        P_spec[idx] = -bus_data[2] / baseMVA
        Q_spec[idx] = -bus_data[3] / baseMVA

# Función para calcular la potencia activa y reactiva inyectada en cada barra
def calc_power(V, theta, Ybus):
    n = len(V)
    P = np.zeros(n)
    Q = np.zeros(n)
    for i in range(n):
        for j in range(n):
            P[i] += V[i] * V[j] * (Ybus[i, j].real * np.cos(theta[i] - theta[j]) + Ybus[i, j].imag * np.sin(theta[i] - theta[j]))
            Q[i] += V[i] * V[j] * (Ybus[i, j].real * np.sin(theta[i] - theta[j]) - Ybus[i, j].imag * np.cos(theta[i] - theta[j]))
    return P, Q

# Función para calcular el Jacobiano
def calc_jacobian(V, theta, Ybus):
    n = len(V)
    J = np.zeros((2*n, 2*n))
    for i in range(n):
        for j in range(n):
            if i == j:
                J[i, j] = -Q_spec[i] - V[i]**2 * Ybus[i, i].imag
                J[i, n+j] = P_spec[i] / V[i] + V[i] * Ybus[i, i].real
                J[n+i, j] = P_spec[i] - V[i]**2 * Ybus[i, i].real
                J[n+i, n+j] = Q_spec[i] / V[i] - V[i] * Ybus[i, i].imag
            else:
                J[i, j] = V[i] * V[j] * (Ybus[i, j].real * np.sin(theta[i] - theta[j]) - Ybus[i, j].imag * np.cos(theta[i] - theta[j]))
                J[i, n+j] = V[i] * (Ybus[i, j].real * np.cos(theta[i] - theta[j]) + Ybus[i, j].imag * np.sin(theta[i] - theta[j]))
                J[n+i, j] = V[i] * V[j] * (Ybus[i, j].real * np.cos(theta[i] - theta[j]) + Ybus[i, j].imag * np.sin(theta[i] - theta[j]))
                J[n+i, n+j] = V[i] * (Ybus[i, j].real * np.sin(theta[i] - theta[j]) - Ybus[i, j].imag * np.cos(theta[i] - theta[j]))
    return J

# Método de Newton-Raphson
def newton_raphson(V, theta, Ybus, P_spec, Q_spec, tol=1e-3, max_iter=10000):
    n = len(V)
    for _ in range(max_iter):
        P, Q = calc_power(V, theta, Ybus)
        dP = P_spec - P
        dQ = Q_spec - Q
        dPQ = np.concatenate((dP, dQ))
        if np.linalg.norm(dPQ) < tol:
            break
        J = calc_jacobian(V, theta, Ybus)
        dX = np.linalg.solve(J, dPQ)
        theta += dX[:n]
        V += dX[n:]
    return V, theta

# Ejecutar Newton-Raphson
V, theta = newton_raphson(V, theta, Ybus, P_spec, Q_spec)

# Mostrar resultados de Newton-Raphson
print("Resultados de Newton-Raphson:")
print("Voltajes (V):", V)
print("Ángulos (theta):", theta)

# Mostrar resultados de pandapower
print("Resultados de pandapower:")
print(red2.res_bus)


   vm_pu  va_degree      p_mw  q_mvar
0 1.0000     0.0000 -212.3102  5.8073
1 1.0334   -14.2765    0.0000  0.0000
2 1.0315   -14.4923   30.0000 20.0000
3 1.0324   -14.4698   15.0000 10.0000
4 1.0292   -14.7303   52.5000 35.0000
5 1.0274   -14.8735   90.0000 60.0000
6 1.0284   -14.8649   22.5000 15.0000
   p_from_mw  q_from_mvar   p_to_mw  q_to_mvar  pl_mw  ql_mvar  i_from_ka  \
0   116.8068     -22.3856 -116.5426     2.5430 0.2642 -19.8426     0.3020   
1    94.2749     -37.5516  -94.0903    17.5655 0.1846 -19.9860     0.2577   
2    86.5426     -22.5430  -86.3250    -7.3809 0.2176 -29.9238     0.2275   
3    33.8250     -27.6191  -33.7786   -12.5423 0.0463 -40.1614     0.1113   
4    79.0903     -27.5655  -78.7297   -32.3478 0.3606 -59.9134     0.2129   
5   -11.2703     -27.6522   11.2786    -2.4577 0.0083 -30.1099     0.0763   

   i_to_ka   i_ka  vm_from_pu  va_from_degree  vm_to_pu  va_to_degree  \
0   0.2966 0.3020      1.0334        -14.2765    1.0315      -14.4923   
1   0.2433